# Paso 5. Convección Lineal Bidimensional

$$
\frac{\partial u}{\partial t} + c \frac{\partial u}{\partial x} + c \frac{\partial u}{\partial y} = 0
$$

Utilizemos la notación en operadores solo para irnos acostumbrando a ellos.

$$
\frac{\partial u}{\partial t} + (c \cdot \nabla)u = 0
$$

Como ahora tenemos derivadas parciales y un espacio bidimensional la discretización debera hacerce a lo largo de ambos ejes $x$ y $y$. Para la discretización de las derivadas parciales, simplemente tomamos la propia definición de derivada parcial, es decir que para la parcial de $x$ solo se debe variar $x$ y dejar fijo el eje $y$.

$$
\frac{\partial u}{\partial x}(i,j) = \frac{u(i+1, j) - u(i, j)}{ \Delta x}
$$

$$
\frac{\partial u}{\partial y}(i,j) = \frac{u(i, j+1) - u(i, j)}{ \Delta y}
$$

Para la derivada de segundo orden.

$$
\frac{\partial^2 u}{\partial x^2}(i,j) = \frac{u(i+1, j) - 2u(i, j) + u(i-1, j)}{ \Delta x^2}
$$

Con las anteriores ecuaciones, podemos definir la discretización de la Convección Lineal Bidimensional. Usando el esquema FT-BS

$$
\frac{u(i,j)_{n+1} - u(i,j)_n}{\Delta t} + c \frac{u(i, j)_n - u(i-1, j)_n}{ \Delta x} + c \frac{u(i, j)_n - u(i, j-1)_n}{ \Delta y} = 0
$$

Resolviendo para la incognita $u(i,j)_{n+1}$

$$
u(i,j)_{n+1} = u(i,j)_n - c \frac{\Delta t}{\Delta x}\Big[ u(i, j)_n - u(i-1, j)_n\Big] - c \frac{\Delta t}{\Delta y}\Big[ u(i, j)_n - u(i, j-1)_n\Big]
$$

In [66]:
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import animation
from mpl_toolkits.mplot3d import Axes3D

def update(i, profile):
    #store the actual state of the array
    u_n = u;
    
    #update array
    #for i in range(1, partitions):
    #    for j in range(1, partitions):
    #        u[i][j] = u_n[i][j] - (c*(dt/dx)) * (u_n[i][j] - u_n[i-1][j]) - (c*(dt/dx)) * (u_n[i][j] - u_n[i][j-1]);    
    # this code does the same showed in comment above but in a more pythonic way
    u[1:,1:] = u_n[1:,1:] - (c*(dt/dx)) * (u_n[1:,1:] - np.roll(u, 1, axis=1)[1:, 1:]) - (c*(dt/dx)) * (u_n[1:,1:] - np.roll(u, 1, axis=0)[1:, 1:])
    
    ax.clear();
    ax.set_zlim(0.0, 0.16);
    ax.grid(False);

    profile = ax.plot_surface(x, y, u);
    
    #Animate Point of view.
    #ax.view_init(elev=15., azim=i)
    return profile,


# How much we want to advance in time?
# at 30 fps if we want simulate 10 seconds => 30*10
t_seconds = 30 * 10;

# The time step size 
dt = 0.005;

# x velocity
c = 7.0;

partitions = 101;

data_x = np.linspace(-3.0, 8.0, num=partitions, retstep=True);
x  = data_x[0]
dx = data_x[1];

data_y = np.linspace(-3.0, 8.0, num=partitions, retstep=True);
y  = data_x[0]
dy = data_x[1];

x, y = np.meshgrid(x, y)
#parameters for 2D normal distribution
sigma = np.ones(2) * 1;
miu   = np.ones(2) * 0.01;
rho = 0.0;

A = 2*np.pi*sigma[0]*sigma[1]*np.sqrt(1-rho**2);
B = 2*(1-rho**2);
C = (x-miu[0])**2 / sigma[0]**2;
D = (y-miu[1])**2 / sigma[1]**2;
E = (2*rho*(x-miu[0])*(y-miu[1])) / (miu[0]*miu[1]);

u = 1 / A * np.exp( -1/B * ( C + D - E ) );

# Create a figure and a 3D Axes
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d');
ax.set_zlim(0, 0.16);
ax.grid(False);

profile = ax.plot_surface(x, y, u);

# Animate
anim = animation.FuncAnimation(fig, update, fargs=(profile,), frames=t_seconds, blit=False)
anim.save('2D_Linear_Convection_Equation_01.mp4', fps=30, writer="ffmpeg", codec="libx264");
plt.close(fig);

In [1]:
# You must have configured ffmpeg in your machine to run the code below
from IPython.display import HTML
video = open("2D_Linear_Convection_Equation_01.mp4", "rb").read()
video_encoded = video.encode("base64")
video_tag = '<video controls alt="test" src="data:video/x-m4v;base64,{0}">'.format(video_encoded)
HTML(video_tag)